In [1]:
# Load autoreload extension
%load_ext autoreload

# Set autoreload behavior
%autoreload 2
import os
import time
import argparse
import random
import numpy as np
from models.ProTACT import build_ProTACT
import tensorflow as tf
from configs.configs import Configs
from utils.read_data_pr import read_pos_vocab, read_word_vocab, read_prompts_we, read_essays_prompts, read_prompts_pos
from utils.general_utils import get_scaled_down_scores, pad_hierarchical_text_sequences, get_attribute_masks, load_word_embedding_dict, build_embedd_table
from evaluators.multitask_evaluator_all_attributes import Evaluator as AllAttEvaluator
from tensorflow import keras
import matplotlib.pyplot as plt
    

[nltk_data] Downloading package punkt to /Users/joohwan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joohwan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# parser = argparse.ArgumentParser(description="ProTACT model")
# parser.add_argument('--test_prompt_id', type=int, default=1, help='prompt id of test essay set')
# parser.add_argument('--seed', type=int, default=12, help='set random seed')
# parser.add_argument('--model_name', type=str,
#                     choices=['ProTACT'],
#                     help='name of model')
# parser.add_argument('--num_heads', type=int, default=2, help='set the number of heads in Multihead Attention')
# parser.add_argument('--features_path', type=str, default='data/hand_crafted_v3.csv')

test_prompt_id = 1
seed = 1
num_heads = 2
features_path = '../data/hand_crafted_v3.csv'

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

print("Test prompt id is {} of type {}".format(test_prompt_id, type(test_prompt_id)))
print("Seed: {}".format(seed))

configs = Configs()

data_path = configs.DATA_PATH
train_path = data_path + str(test_prompt_id) + '/train.pk'
dev_path = data_path + str(test_prompt_id) + '/dev.pk'
test_path = data_path + str(test_prompt_id) + '/test.pk'
pretrained_embedding = configs.PRETRAINED_EMBEDDING
embedding_path = configs.EMBEDDING_PATH
readability_path = configs.READABILITY_PATH
prompt_path = configs.PROMPT_PATH
vocab_size = configs.VOCAB_SIZE
epochs = configs.EPOCHS
batch_size = configs.BATCH_SIZE
print("Numhead : ", num_heads, " | Features : ", features_path, " | Pos_emb : ", configs.EMBEDDING_DIM)

read_configs = {
    'train_path': train_path,
    'dev_path': dev_path,
    'test_path': test_path,
    'features_path': features_path,
    'readability_path': readability_path,
    'vocab_size': vocab_size
}

Test prompt id is 1 of type <class 'int'>
Seed: 1
Numhead :  2  | Features :  ../data/hand_crafted_v3.csv  | Pos_emb :  50


In [3]:
pos_vocab = read_pos_vocab(read_configs)
# read POS for prompts
prompt_pos_data = read_prompts_pos(prompt_path, pos_vocab) # for prompt POS embedding 

word_vocab = read_word_vocab(read_configs)
# read words for prompts 
prompt_data = read_prompts_we(prompt_path, word_vocab) # for prompt word embedding 

# read essays and prompts
train_data, dev_data, test_data = read_essays_prompts(read_configs, prompt_data, prompt_pos_data, pos_vocab) 

if pretrained_embedding:
    embedd_dict, embedd_dim, _ = load_word_embedding_dict(embedding_path)
    embedd_matrix = build_embedd_table(word_vocab, embedd_dict, embedd_dim, caseless=True)
    embed_table = [embedd_matrix]
else:
    embed_table = None

 prompt_pos size: 8
 prompt_words size: 8
 pos_x size: 9513
 readability_x size: 9513
 pos_x size: 1680
 readability_x size: 1680
 pos_x size: 1783
 readability_x size: 1783
Loading GloVe ...
OOV number =189, OOV ratio = 0.047262


In [ ]:
import pandas as pd
print(len(train_data['features_x'][0]))
pd.DataFrame(train_data)
# embedd_dict

In [4]:
max_sentlen = max(train_data['max_sentlen'], dev_data['max_sentlen'], test_data['max_sentlen'])
max_sentnum = max(train_data['max_sentnum'], dev_data['max_sentnum'], test_data['max_sentnum'])
prompt_max_sentlen = prompt_data['max_sentlen']
prompt_max_sentnum = prompt_data['max_sentnum']

print('max sent length: {}'.format(max_sentlen))
print('max sent num: {}'.format(max_sentnum))
print('max prompt sent length: {}'.format(prompt_max_sentlen))
print('max prompt sent num: {}'.format(prompt_max_sentnum))

train_data['y_scaled'] = get_scaled_down_scores(train_data['data_y'], train_data['prompt_ids'])
dev_data['y_scaled'] = get_scaled_down_scores(dev_data['data_y'], dev_data['prompt_ids'])
test_data['y_scaled'] = get_scaled_down_scores(test_data['data_y'], test_data['prompt_ids'])

X_train_pos = pad_hierarchical_text_sequences(train_data['pos_x'], max_sentnum, max_sentlen)
X_dev_pos = pad_hierarchical_text_sequences(dev_data['pos_x'], max_sentnum, max_sentlen)
X_test_pos = pad_hierarchical_text_sequences(test_data['pos_x'], max_sentnum, max_sentlen)

X_train_pos = X_train_pos.reshape((X_train_pos.shape[0], X_train_pos.shape[1] * X_train_pos.shape[2]))
X_dev_pos = X_dev_pos.reshape((X_dev_pos.shape[0], X_dev_pos.shape[1] * X_dev_pos.shape[2]))
X_test_pos = X_test_pos.reshape((X_test_pos.shape[0], X_test_pos.shape[1] * X_test_pos.shape[2]))

X_train_prompt = pad_hierarchical_text_sequences(train_data['prompt_words'], max_sentnum, max_sentlen)
X_dev_prompt = pad_hierarchical_text_sequences(dev_data['prompt_words'], max_sentnum, max_sentlen)
X_test_prompt = pad_hierarchical_text_sequences(test_data['prompt_words'], max_sentnum, max_sentlen)

X_train_prompt = X_train_prompt.reshape((X_train_prompt.shape[0], X_train_prompt.shape[1] * X_train_prompt.shape[2]))
X_dev_prompt = X_dev_prompt.reshape((X_dev_prompt.shape[0], X_dev_prompt.shape[1] * X_dev_prompt.shape[2]))
X_test_prompt = X_test_prompt.reshape((X_test_prompt.shape[0], X_test_prompt.shape[1] * X_test_prompt.shape[2]))

X_train_prompt_pos = pad_hierarchical_text_sequences(train_data['prompt_pos'], max_sentnum, max_sentlen)
X_dev_prompt_pos = pad_hierarchical_text_sequences(dev_data['prompt_pos'], max_sentnum, max_sentlen)
X_test_prompt_pos = pad_hierarchical_text_sequences(test_data['prompt_pos'], max_sentnum, max_sentlen)

X_train_prompt_pos = X_train_prompt_pos.reshape((X_train_prompt_pos.shape[0], X_train_prompt_pos.shape[1] * X_train_prompt_pos.shape[2]))
X_dev_prompt_pos = X_dev_prompt_pos.reshape((X_dev_prompt_pos.shape[0], X_dev_prompt_pos.shape[1] * X_dev_prompt_pos.shape[2]))
X_test_prompt_pos = X_test_prompt_pos.reshape((X_test_prompt_pos.shape[0], X_test_prompt_pos.shape[1] * X_test_prompt_pos.shape[2]))

X_train_linguistic_features = np.array(train_data['features_x'])
X_dev_linguistic_features = np.array(dev_data['features_x'])
X_test_linguistic_features = np.array(test_data['features_x'])

X_train_readability = np.array(train_data['readability_x'])
X_dev_readability = np.array(dev_data['readability_x'])
X_test_readability = np.array(test_data['readability_x'])

Y_train = np.array(train_data['y_scaled'])
Y_dev = np.array(dev_data['y_scaled'])
Y_test = np.array(test_data['y_scaled'])

X_train_attribute_rel = get_attribute_masks(Y_train)
X_dev_attribute_rel = get_attribute_masks(Y_dev)
X_test_attribute_rel = get_attribute_masks(Y_test)

print('================================')
print('X_train_pos: ', X_train_pos.shape)
print('X_train_prompt_words: ', X_train_prompt.shape)
print('X_train_prompt_pos: ', X_train_prompt_pos.shape)
print('X_train_readability: ', X_train_readability.shape)
print('X_train_ling: ', X_train_linguistic_features.shape)
print('X_train_attribute_rel: ', X_train_attribute_rel.shape)
print('Y_train: ', Y_train.shape)

print('================================')
print('X_dev_pos: ', X_dev_pos.shape)
print('X_dev_prompt_words: ', X_dev_prompt.shape)
print('X_dev_prompt_pos: ', X_dev_prompt_pos.shape)
print('X_dev_readability: ', X_dev_readability.shape)
print('X_dev_ling: ', X_dev_linguistic_features.shape)
print('X_dev_attribute_rel: ', X_dev_attribute_rel.shape)
print('Y_dev: ', Y_dev.shape)

print('================================')
print('X_test_pos: ', X_test_pos.shape)
print('X_test_prompt_words: ', X_test_prompt.shape)
print('X_test_prompt_pos: ', X_test_prompt_pos.shape)
print('X_test_readability: ', X_test_readability.shape)
print('X_test_ling: ', X_test_linguistic_features.shape)
print('X_test_attribute_rel: ', X_test_attribute_rel.shape)
print('Y_test: ', Y_test.shape)
print('================================')

max sent length: 50
max sent num: 97
max prompt sent length: 18
max prompt sent num: 8
X_train_pos:  (9513, 4850)
X_train_prompt_words:  (9513, 4850)
X_train_prompt_pos:  (9513, 4850)
X_train_readability:  (9513, 35)
X_train_ling:  (9513, 51)
X_train_attribute_rel:  (9513, 9)
Y_train:  (9513, 9)
X_dev_pos:  (1680, 4850)
X_dev_prompt_words:  (1680, 4850)
X_dev_prompt_pos:  (1680, 4850)
X_dev_readability:  (1680, 35)
X_dev_ling:  (1680, 51)
X_dev_attribute_rel:  (1680, 9)
Y_dev:  (1680, 9)
X_test_pos:  (1783, 4850)
X_test_prompt_words:  (1783, 4850)
X_test_prompt_pos:  (1783, 4850)
X_test_readability:  (1783, 35)
X_test_ling:  (1783, 51)
X_test_attribute_rel:  (1783, 9)
Y_test:  (1783, 9)


In [ ]:
pd.DataFrame(train_data)

In [ ]:
# train_features_list = [X_train_pos, X_train_prompt, X_train_prompt_pos, X_train_linguistic_features, X_train_readability]
X_train_pos

In [ ]:
Y_train[0]

In [ ]:
import keras
print(keras.__version__)

In [5]:
train_features_list = [X_train_pos, X_train_prompt, X_train_prompt_pos, X_train_linguistic_features, X_train_readability]
dev_features_list = [X_dev_pos, X_dev_prompt, X_dev_prompt_pos, X_dev_linguistic_features, X_dev_readability]
test_features_list = [X_test_pos, X_test_prompt, X_test_prompt_pos, X_test_linguistic_features, X_test_readability]

model = build_ProTACT(len(pos_vocab), len(word_vocab), max_sentnum, max_sentlen, 
                  X_train_readability.shape[1],
                  X_train_linguistic_features.shape[1],
                  configs, Y_train.shape[1], num_heads, embed_table)

evaluator = AllAttEvaluator(test_prompt_id, dev_data['prompt_ids'], test_data['prompt_ids'], dev_features_list,
                            test_features_list, Y_dev, Y_test, seed)

evaluator.evaluate(model, -1, print_info=True)

/Users/joohwan/miniforge3/envs/jup/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/Users/joohwan/miniforge3/envs/jup/lib/python3.9/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'multi_head_attention', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/Users/joohwan/miniforge3/envs/jup/lib/python3.9/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'multi_head_attention_10', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ prompt_word_input   │ (None, 4850)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prompt_pos_input    │ (None, 4850)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_input           │ (None, 4850)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prompt (Embedding)  │ (None, 4850, 50)  │    200,000 │ prompt_word_inpu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 4850)      │          0 │ prompt_word_inpu… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_prompt          │ (None, 4850, 50)  │      1,800 │ prompt_pos_input… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 4850)      │          0 │ prompt_pos_input… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_x (Embedding)   │ (None, 4850, 50)  │      1,800 │ pos_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 4850)      │          0 │ pos_input[0][0]   │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prompt_maskedout    │ (None, 4850, 50)  │          0 │ prompt[0][0],     │
│ (ZeroMaskedEntries) │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prompt_pos_maskedo… │ (None, 4850, 50)  │          0 │ pos_prompt[0][0], │
│ (ZeroMaskedEntries) │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_x_maskedout     │ (None, 4850, 50)  │          0 │ pos_x[0][0],      │
│ (ZeroMaskedEntries) │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 4850, 50)  │          0 │ prompt_maskedout… │
│                     │                   │            │ prompt_pos_maske… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_drop_x          │ (None, 4850, 50)  │          0 │ pos_x_maskedout[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prompt_drop_x       │ (None, 4850, 50)  │          0 │ add_2[0][0]       │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_resh_W          │ (None, 97, 50,    │          0 │ pos_drop_x[0][0]  │
│ (Reshape)           │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prompt_resh_W       │ (None, 97, 50,    │          0 │ prompt_drop_x[0]… │
│ (Reshape)           │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,636,557 (6.24 MB)

 Trainable params: 1,636,557 (6.24 MB)

 Non-trainable params: 0 (0.00 B)

53/53 ━━━━━━━━━━━━━━━━━━━━ 48s 507ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 237ms/step
CURRENT EPOCH: -1
[DEV] AVG QWK: -0.003
[DEV] score QWK: -0.026
[DEV] content QWK: -0.037
[DEV] organization QWK: 0.047
[DEV] word_choice QWK: 0.161
[DEV] sentence_fluency QWK: 0.065
[DEV] conventions QWK: -0.049
[DEV] prompt_adherence QWK: -0.081
[DEV] language QWK: -0.038
[DEV] narrativity QWK: -0.065
------------------------
[TEST] AVG QWK: -0.048
[TEST] score QWK: 0.0
[TEST] content QWK: -0.21
[TEST] organization QWK: -0.083
[TEST] word_choice QWK: 0.022
[TEST] sentence_fluency QWK: 0.027
[TEST] conventions QWK: -0.047
------------------------
[BEST TEST] AVG QWK: -0.048, {epoch}: -1
[BEST TEST] score QWK: 0.0
[BEST TEST] content QWK: -0.21
[BEST TEST] organization QWK: -0.083
[BEST TEST] word_choice QWK: 0.022
[BEST TEST] sentence_fluency QWK: 0.027
[BEST TEST] conventions QWK: -0.047
-------------------------------------------------------------------------------------------------------------------

In [ ]:
model.summary()

In [ ]:
# class CustomHistory(keras.callbacks.Callback):
#     def init(self):
#         self.train_loss = []
#         self.val_loss = []
#         self.train_acc = []
#         self.val_acc = []        
        
#     def on_epoch_end(self, batch, logs={}):
#         self.train_loss.append(logs.get('loss'))
#         self.val_loss.append(logs.get('val_loss'))
#         self.train_acc.append(logs.get('acc'))
#         self.val_acc.append(logs.get('val_acc'))
# custom_hist = CustomHistory()
# custom_hist.init() 

#  for ii in range(epochs):
#     print('Epoch %s/%s' % (str(ii + 1), epochs))
#     start_time = time.time()
#     model.fit(
#         train_features_list,
#         Y_train, batch_size=batch_size, epochs=5, verbose=0, shuffle=True, validation_data=(dev_features_list,Y_dev),callbacks=[custom_hist,checkpoint])
#     tt_time = time.time() - start_time
#     print("Training one epoch in %.3f s" % tt_time)
#     evaluator.evaluate(model, ii + 1)
#     print("Train Loss: ", custom_hist.train_loss[-1], "|| Val Loss: ", custom_hist.val_loss[-1])

# evaluator.print_final_info()

'''# show the loss as the graph
fig, loss_graph = plt.subplots()
loss_graph.plot(custom_hist.train_loss,'y',label='train loss')
loss_graph.plot(custom_hist.val_loss,'r',label='val loss')
loss_graph.set_xlabel('epoch')
loss_graph.set_ylabel('loss')
plt.savefig(str('images/protact/test_prompt_'+ str(test_prompt_id) + '_seed_' + str(seed) + '_loss.png'))'''

In [6]:
# 저장한 체크포인트 있다면: 이어서 학습 시작할때 
# Checkpoint 폴더 안에 있는 .h5 파일 지울것
# model.load_weights('Checkpoint/tensor{epoch}')

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    # epoch 마다 파일명 다르게 저장
    filepath='Checkpoint/bestmodel{epoch}.weights.h5',

    # epoch 마다 weights 들만 저장
    save_freq='epoch',
    save_weights_only = True,

    # validation accruary 가 최대일때만 저장 
    monitor='val_loss',
    mode='min'
)

In [ ]:
class CustomHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.train_loss = []
        self.val_loss = []
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        epoch_time = time.time() - self.start_time
        self.epoch_times.append(epoch_time)
        print(f"Epoch {epoch + 1}: Train Loss: {logs.get('loss')} || Val Loss: {logs.get('val_loss')}")
        print(f"Epoch {epoch + 1} completed in {epoch_time:.3f} seconds")

        # Evaluate the model (you might need to adjust this to your specific evaluation function)
        evaluator.evaluate(self.model, epoch + 1)

custom_hist = CustomHistory()
model.fit(
    train_features_list,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    shuffle=True,
    validation_data=(dev_features_list, Y_dev),
    callbacks=[custom_hist, checkpoint]
)

Epoch 1/50
 78/952 ━━━━━━━━━━━━━━━━━━━━ 24:06 2s/step - loss: 0.0284

In [ ]:
# 실행 X
# TEST: 위에서 진행된곳 까지 결과가 같다.
model.load_weights('Checkpoint/bestmodel1.h5')
evaluator.evaluate(model,1)